In [ ]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
# from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter


print("Pipeline SDK-specific imports completed")

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore
def_blob_store = ws.get_default_datastore() 
# The following call GETS the Azure Blob Store associated with your workspace.
# Note that workspaceblobstore is **the name of this store and CANNOT BE CHANGED and must be used as is** 
def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))

In [ ]:
cts = ws.compute_targets
for ct in cts:
    print(ct)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

aml_compute_target = "cpu-cluster-2"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "Standard_DS3_v2",
                                                                min_nodes = 0, 
                                                                max_nodes = 1)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

In [ ]:
# Uses default values for PythonScriptStep construct.

source_directory = '../code/scripts'
os.mkdir(source_directory)
print("Path is created")
print('Source directory for the step is {}.'.format(os.path.realpath(source_directory)))

In [ ]:
%%writefile $source_directory/script.py
import argparse
import os

print("In script.py")
print("As a data scientist, this is where I use my training code.")

parser = argparse.ArgumentParser("train")

parser.add_argument("--pipeline_arg", type=str, help="pipeline_arg")
# parser.add_argument("--output_train", type=str, help="output_train directory")

args = parser.parse_args()

print("Argument 1: %s" % args.pipeline_arg)
# print("Argument 2: %s" % args.output_train)

# if not (args.output_train is None):
#     os.makedirs(args.output_train, exist_ok=True)
#     print("%s created" % args.output_train)

In [ ]:
# Data path parameter
data_path = DataPath(datastore=def_blob_store, path_on_datastore='sample_datapath1')
datapath1_pipeline_param = PipelineParameter(name="input_datapath", default_value=data_path)
datapath_input = (datapath1_pipeline_param, DataPathComputeBinding(mode='moun

# String Parameter
pipeline_param = PipelineParameter(name="pipeline_arg", default_value="default_val")


step1 = PythonScriptStep(name="script_step",
                         script_name="script.py",
                         arguments=["--pipeline_arg", pipeline_param],
                         compute_target=aml_compute, 
                         source_directory=source_directory,
                         allow_reuse=True)
print("Step1 created")

In [ ]:
steps = [step1]
print("Step lists created")

In [ ]:
pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built")

In [ ]:
pipeline1.validate()
print("Pipeline validation complete")

In [ ]:
# pipeline_run1 = Experiment(ws, 'pipeline1-submit').submit(pipeline1, regenerate_outputs=False, \
#                                                           pipeline_parameters={'pipeline_arg':'hello pipeline param'})
# print("Pipeline is submitted for execution")

In [ ]:
# RunDetails(pipeline_run1).show()

In [ ]:
published_pipeline = pipeline1.publish(name="ParamPipeline", description="Pipeline to test parameters", continue_on_step_failure=True)
published_pipeline

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name="ParamPipelineEndpoint",
                                            pipeline=pipeline1, description="Test description Notebook")